In [2]:
from time import sleep
import random

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import re

import requests
import pandas as pd
import json

# ---------------------------------------
# Path
import sys
sys.path.append('../')

import warnings
#warnings.filterwarnings('ignore')

from src.support import get_dotd, get_df_dotd, get_df_circuit, get_df_drivers

### Driver of the day

In [ ]:
try:
    df_dotd = pd.read_csv('../data/dotd.csv')
except FileNotFoundError:
    soup = get_dotd()
    df_dotd = get_df_dotd(soup)
    df_dotd.to_csv('../data/dotd.csv', index = False)

### Circuits (2023)

In [ ]:
try:
    df_circuit = pd.read_csv('../data/circuit.csv')
except FileNotFoundError:
    df_circuit = get_df_circuit(2023)
    df_circuit.to_csv('../data/circuit.csv', index = False)

### Drivers (2023)

In [5]:
try:
    df_drivers = pd.read_csv('../data/drivers.csv')
except FileNotFoundError:
    df_drivers = get_df_drivers(2023)
    df_drivers.to_csv('../data/drivers.csv', index = False)